---
## Studying Artificial Neural Networks 
Hello! Welcome to my notes on Artificial Neural Networks. This notebook is based on material available on [Deep Learning A-Z: Hands-On Artificial Neural Networks](https://www.udemy.com/course/deeplearning/?utm_source=adwords&utm_medium=udemyads&utm_campaign=LongTail_la.EN_cc.BR&utm_content=deal4584&utm_term=_._ag_112130202440_._ad_467215026440_._kw__._de_c_._dm__._pl__._ti_dsa-1007766171312_._li_1031562_._pd__._&matchtype=b&gclid=Cj0KCQiAx9mABhD0ARIsAEfpavTqbFhHyVvzjw8I9ezj2OsxiL2cDipfPahalnBmoijpYRfup0p-EqoaArEbEALw_wcB). It is a **fantastic** course!

Our goal is to predict whether a bank customer, having their data as features, will stay in the bank or not.

*by: Pedro Florencio*

---

#### Importing the libraries

In [50]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [51]:
tf.__version__ 

'2.4.1'

#### Step 1 - Data Preprocessing

In [52]:
df = pd.read_csv('Churn_Modelling.csv') # Importing dataset
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [53]:
X = df.iloc[:, 3:-1].values # The first 3 columns (RowNumber, CustomerId and Surname) have no impact on our modeling
y = df.iloc[:,-1].values

##### Encoding categorical data

In [54]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2]) # Female = 0 and Male = 1
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

One Hot encoding the "Geography" column

In [55]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X) # France = [1 0 0] Spain = [0 0 1] Germany = [0 1 0]

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


##### Splitting the dataset into the Training set and Test set

In [56]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state = 0) 

##### Feature Scaling

In [57]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform (X_test)

#### Step 2 - Building the ANN

In [58]:
ann = tf.keras.models.Sequential()

##### Adding the input layer and the first hidden layer

In [59]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

##### Adding the second hidden layer

In [60]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

##### Building the output layer

In [61]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid')) # We just need a neuron

#### Step 3 - Training the ANN

##### Compiling the ANN

In [62]:
ann.compile(optimizer = 'adam',loss = 'binary_crossentropy', metrics = ['accuracy'])

##### Training the ANN on the Training set

In [63]:
ann.fit(X_train,y_train,batch_size=32,epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5567 - accuracy: 0.7897
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4612 - accuracy: 0.8083
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4405 - accuracy: 0.8073
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4327 - accuracy: 0.8157
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4252 - accuracy: 0.8143
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4150 - accuracy: 0.8171
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4198 - accuracy: 0.8186
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4155 - accuracy: 0.8159
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4251 - accuracy: 0.8155
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4014 - accura

#### Step 4 - Making the predictions and evaluating the model

##### Predicting the result of a single observation

**Using our ANN model we will predict whether the customer with the following informations will leave the bank:**

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer ?

In [64]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]])))

[[0.02433643]]


In [65]:
print(ann.predict(sc.transform([[1,0,0,600,1,40,3,60000,2,1,1,50000]]))> 0.5)

[[False]]


3.136 % to this customer leaves the bank

#### Predicting the test results

In [68]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Predicted | Real

#### Making the Confusion Matrix

In [67]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix (y_test, y_pred)
print(cm)
accuracy_score(y_test,y_pred)

[[1512   83]
 [ 204  201]]


0.8565

* 1517 - Correct predictions that the customers stay in the bank

* 200 - Incorrect predictions that the customers stay in the bank


* 205 - Correct predictions that the customers leave the bank

* 78 - Incorrect predictions that the customers leave the bank


**Thank you!**

---